In [20]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import KDTree

In [21]:
def merge_naive(pred_1, pred_2, cutoff=20):
    """
    naive cluster merging:
    iterate over hits; if a hit belongs to a larger cluster in pred_2, it is reassigned
    """
    if pred_1 is None:
        return pred_2
    c1, c2 = Counter(pred_1), Counter(pred_2)  # track id -> track size
    n1, n2 = np.vectorize(c1.__getitem__)(pred_1), np.vectorize(
        c2.__getitem__)(pred_2)  # hit id -> track size
    pred = pred_1.copy()
    idx = (n2 > n1) & (n2 < cutoff)
    pred[idx] = max(pred_1) + 1 + pred_2[idx]
    return label_encode(pred)

In [23]:
df=pd.read_csv("submission.csv")
df2=pd.read_csv("hough2.csv")
df=df[['event_id', 'hit_id', 'track_id']]
df2=df2[['event_id', 'hit_id', 'track_id']]

df['track_id']=merge(df['track_id'],df2['track_id'])
df=df[['event_id', 'hit_id', 'track_id']]

In [25]:
root="/home/alexanderliao/data/Kaggle/competitions/trackml-particle-identification/test/"

for i in tqdm(range(125)):
    if i==0:
        hits=pd.read_csv(root+"event"+str(i).zfill(9)+"-hits.csv")
    else:
        temp=pd.read_csv(root+"event"+str(i).zfill(9)+"-hits.csv")
        hits=pd.concat([hits,temp])

100%|██████████| 125/125 [01:12<00:00,  1.72it/s]


In [26]:
hits[['hit_id', 'x','y','z']].to_csv("feature.csv",index=False)

In [27]:
hits=hits[['hit_id','x','y','z']]

In [31]:
index = 0
for i in tqdm(range(125)):
    event = df.loc[df['event_id'] == i]
    temp = pd.merge(event,hits.iloc[index:index+event.shape[0]],on='hit_id')
    index=index+event.shape[0]
    #print(index)
    temp[['event_id','hit_id', 'track_id','x','y','z']].to_csv("./events/event"+str(i).zfill(3)+".csv",index=False)

100%|██████████| 125/125 [01:17<00:00,  1.61it/s]


In [32]:
index

13741466

In [36]:
def extend(df,limit=0.04, num_neighbours=18):

    #df = submission.merge(hits,  on=['hit_id'], how='left')
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in [x / 10.0 for x in range(-900, 900, 5)]:

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.5)/180*np.pi) & (df.arctan2<(angle+1.5)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<3: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1[['x', 'y', 'z']].values.T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        c = np.cos(a)
        s = np.sin(a)
        #tree = KDTree(np.column_stack([a,r]), metric='euclidean')
        tree = KDTree(np.column_stack([c, s, r]), metric='euclidean')


        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]
            c0 = c[idx0]
            c1 = c[idx1]
            s0 = s[idx0]
            s1 = s[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0)

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1)



            ## extend start point
            ns = tree.query([[c0, s0, r0]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r0 - r[ns], a0 - a[ns])
            diff = 1 - np.cos(direction - direction0)
            ns = ns[(r0 - r[ns] > 0.01) & (diff < (1 - np.cos(limit)))]
            for n in ns: df.loc[df.hit_id == hit_ids[n], 'track_id'] = p

            ## extend end point
            ns = tree.query([[c1, s1, r1]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns] - r1, a[ns] - a1)
            diff = 1 - np.cos(direction - direction1)
            ns = ns[(r[ns] - r1 > 0.01) & (diff < (1 - np.cos(limit)))]
            for n in ns:  df.loc[df.hit_id == hit_ids[n], 'track_id'] = p

    #print ('\r')
    df = df[['event_id', 'hit_id', 'track_id','x','y','z']]
    return df

In [ ]:
for i in tqdm(range(125)):
    result=pd.read_csv("./events/event"+str(i).zfill(3)+".csv")
    #print("before:{}".format(result.shape[0]))
    for j in range(8):
        result=extend(result,limit=0.05, num_neighbours=18)
    #print("after:{}".format(result.shape[0]))
    result[['event_id', 'hit_id', 'track_id']].to_csv("./enhanced/enhanced"+str(i).zfill(3)+".csv",index=False)

  0%|          | 0/125 [00:00<?, ?it/s]

 89.5000000

  1%|          | 1/125 [07:43<15:57:02, 463.09s/it]

 89.5000000

  2%|▏         | 2/125 [15:29<15:53:10, 464.97s/it]

...

 89.5000000

 61%|██████    | 76/125 [8:13:50<5:18:23, 389.88s/it]

 89.5000000

 62%|██████▏   | 77/125 [8:20:12<5:11:49, 389.78s/it]

 -39.000000

In [40]:
#del temp
#del final
for i in tqdm(range(125)):
    temp=pd.read_csv("./enhanced/enhanced"+str(i).zfill(3)+".csv")
    temp=temp[['event_id', 'hit_id', 'track_id']]
    print(temp.shape[0])
    print(i)
    if i==0:
        final=temp
    else:
        final=pd.concat([final,temp])
        #print(final.shape[0])

  3%|▎         | 4/125 [00:00<00:07, 16.20it/s]

119016
0
122319
1
105772
2
109438
3


  7%|▋         | 9/125 [00:00<00:06, 18.99it/s]

119419
4
107928
5
107605
6
116719
7
115088
8


 10%|█         | 13/125 [00:00<00:05, 18.83it/s]

...

116194
121
92200
122


 99%|█████████▉| 124/125 [00:19<00:00,  6.49it/s]

102393
123


100%|██████████| 125/125 [00:19<00:00,  6.47it/s]

108117
124


In [42]:
final[['event_id', 'hit_id', 'track_id']]=final[['event_id', 'hit_id', 'track_id']].fillna(0).astype('int') 
final.to_csv('./enhanced.csv',index=False)


In [ ]:
a=pd.read_csv("enhanced.csv")

In [ ]:
a #sanity checks